In [2]:
print('hello')

hello


In [3]:
import surprise
from collections import defaultdict

from SPARQLWrapper import SPARQLWrapper, JSON
from IPython.display import display, HTML
import pandas as pd
import requests
import re

In [14]:
sparql = SPARQLWrapper("https://query.wikidata.org/sparql", agent='Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36')
API_ENDPOINT = "https://www.wikidata.org/w/api.php"

In [17]:
k = 4
films_count = 5
id = input('Enter user ID: ')

data = surprise.Dataset.load_builtin('ml-100k')

trainset = data.build_full_trainset()
sim_options = {'name': 'cosine', 'user_based': True, 'min_support': films_count}
algo = surprise.KNNBaseline(k=k, sim_options=sim_options)
algo.fit(trainset)

testset = trainset.build_anti_testset()
testset = filter(lambda x: x[0] == id, testset)

predictions = algo.test(testset)

top_n = defaultdict(list)
for uid, iid, _, est, _ in predictions:
    top_n[uid].append((iid, round(est, 3)))

for uid, user_ratings in top_n.items():
    user_ratings.sort(key=lambda x: x[1], reverse=True)
    top_n[uid] = user_ratings[:5]

file_name = surprise.get_dataset_dir() + '/ml-100k/ml-100k/u.item'
item = {}

with open(file_name, 'r') as f:
    for line in f:
        line = line.split('|')
        item[line[0]] = (line[1], line[2])

print(f'User {id}:')

movies = []
for movie_id, rating in top_n[id]:
    film = re.sub(r'[\s]{1}\([0-9]{4}\)', "", str(item[movie_id][0]))
    movies.append(film)
    print(str(movie_id) + "\t" + str(rating) + "\t" + str(item[movie_id]))

Enter user ID: 9
Estimating biases using als...
Computing the cosine similarity matrix...
Done computing similarity matrix.
User 9:
32	5	('Crumb (1994)', '01-Jan-1994')
96	5	('Terminator 2: Judgment Day (1991)', '01-Jan-1991')
430	5	('Duck Soup (1933)', '01-Jan-1933')
169	5	('Wrong Trousers, The (1993)', '01-Jan-1993')
509	5	('My Left Foot (1989)', '01-Jan-1989')


In [15]:
def Sparql(sparql, film):
    
    queryString = """

    SELECT DISTINCT ?person ?personLabel ?film ?filmLabel 
    WHERE {
      wd:"""+ film +""" wdt:P57 ?person.
      ?film wdt:P31 wd:Q11424.
      ?film wdt:P57 ?director
      FILTER(?director = ?person).
      SERVICE wikibase:label { bd:serviceParam wikibase:language "ru,en". }
    }
    """
    sparql.setQuery(queryString)
    sparql.setReturnFormat(JSON)
    results = sparql.query().convert()
    results_df = pd.io.json.json_normalize(results['results']['bindings'])
    return results_df

In [18]:
for movie in movies:
    display(HTML('<h3>'+ movie + ' :</h3>'))
    params = {'action' : 'wbsearchentities','format' : 'json','language' : 'en','search': movie}
    res = requests.get(API_ENDPOINT, params = params)
    try:
        film = res.json()['search'][0]['id']
        print(film)
        res = Sparql(sparql, film);
        res = res.reindex(columns=['personLabel.value', 'filmLabel.value'])
        print(res)
  
    except:
        display(HTML('<p>film not found ¯\_(ツ)_/¯</p>'))

Q2619747
  personLabel.value    filmLabel.value
0     Терри Цвигофф      Мир призраков
1     Терри Цвигофф              Крамб
2     Терри Цвигофф  Реклама для гения
3     Терри Цвигофф       Плохой Санта
4     Терри Цвигофф           Луи Блуи


Q170564
  personLabel.value            filmLabel.value
0    Джеймс Кэмерон                      Чужие
1    Джеймс Кэмерон                 Терминатор
2    Джеймс Кэмерон       Парк юрского периода
3    Джеймс Кэмерон                     Бездна
4    Джеймс Кэмерон             Правдивая ложь
5    Джеймс Кэмерон                 Titanic 3D
6    Джеймс Кэмерон          Пиранья 2: Нерест
7    Джеймс Кэмерон  Терминатор 2: Судный день
8    Джеймс Кэмерон                  Q55728868
9    Джеймс Кэмерон                    Титаник


Q1125566
   personLabel.value      filmLabel.value
0       Маккэри, Лео       Part Time Wife
1       Маккэри, Лео     Идти своим путём
2       Маккэри, Лео         Млечный Путь
3       Маккэри, Лео             Q3205560
4       Маккэри, Лео         Blow by Blow
..               ...                  ...
82      Маккэри, Лео         Jeffries Jr.
83      Маккэри, Лео      Outdoor Pajamas
84      Маккэри, Лео          Sweet Daddy
85      Маккэри, Лео      Too Many Mammas
86      Маккэри, Лео  Why Husbands Go Mad

[87 rows x 2 columns]


Q746574
  personLabel.value      filmLabel.value
0     Шеридан, Джим               Братья
1     Шеридан, Джим                 Поле
2     Шеридан, Джим               Боксёр
3     Шеридан, Джим  Разбогатей или умри
4     Шеридан, Джим             Дом грёз
5     Шеридан, Джим          Во имя отца
6     Шеридан, Джим       Моя левая нога
7     Шеридан, Джим            В Америке
8     Шеридан, Джим      Скрижали судьбы
